In [1]:
import os
import re
import string
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm

import cv2

In [2]:
tqdm.pandas()

/home/andreas/miniconda3/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
data_path = "../data/raw"

In [4]:
d_train = pd.read_csv(os.path.join(data_path, "new_training_set.csv"),
                      usecols=["title_1", "image_1", "title_2", "image_2", "Label"])

In [5]:
d_test = pd.read_csv(os.path.join(data_path, "new_test_sample.csv"),
                    names=["pair_index", "title_1", "image_1", "title_2", "image_2"], skiprows=1)

In [6]:
d_final = pd.read_csv(os.path.join("../data/final", "new_test_set.csv"),
                    names=["pair_index", "title_1", "image_1", "title_2", "image_2"], skiprows=1)

In [7]:
d_final.shape

(32580, 5)

In [8]:
d_train.shape

(10181, 5)

In [9]:
d_test.shape

(207, 5)

In [10]:
d_train.head()

,title_1,image_1,title_2,image_2,Label
0,Johnson’s ® Top to Toe Hair & Body Bath 500ml,fdff8b9b8229da091dd7d070aae05f81.jpg,Johnson's cottontouch top to toe hair & body b...,41e191742760932598c7bd201e5dad47.jpg,0
1,Sandal Humble,906cc44f0be72d4e767669b5b63e3a17.jpg,Sandal Humble Glass - Glanzton,7a556b836bfdd08ea592216440524a34.jpg,0
2,PROMO LIKUID LIKUIT LIQUIT BABY POD LIQUID SAL...,475c26635de18b9f93032400732ff336.jpg,Voporizer Liquit - Likuit - Likuid - Liquid Pr...,ace93bec689f3f1565800c500a8341fa.jpg,0
3,6 Pasang / Set Anting Tusuk Bentuk Lingkaran A...,e630997f6217555d6026547ad1c15f0b.jpg,Subei 6 Pasang / Set Anting Tusuk Boho Bohemia...,31abbc176b09f5bd1728cfc3ecbbfb9c.jpg,0
4,ROREC NATURAL SKIN CARE MASK ROREC SHEET MASK ...,a27d11700a7902febd039dc3a96f10f2.jpg,Rorec 86 Natural Skin Care Shert Mask All Variant,813ad9dd638c10f1765db9dde20c9e42.jpg,1


In [11]:
d_test.head()

,pair_index,title_1,image_1,title_2,image_2
0,0,12.12 SUPER PROMO !! Sandal Jepit Pantai Fashion,83d1798fee1c90c2845204d9261169bb.jpg,Clarisse CRAZY OFFER Beli 5 Dapet 12 POLKA SUM...,caba83a8a7f9def9c4d268b6c34da7f4.jpg
1,1,Damai fashion jakarta - long dress JUMBO wanit...,126868769ca4a4694d36d28960f9de8a.jpg,[VIP] kasih fashion jakarta - long dress JUMBO...,7fdfe855a7be9c87238757c43b712b81.jpg
2,2,My Baby Minyak Telon 145 ML,86aee3dc281911f5f9d50fea17b978f0.jpg,My Baby Minyak Telon Plus 145 Ml 4btl kemasan ...,0ec544d3d4169df76ae156e76c724f0c.jpg
3,3,Creative Waterborne Marker Very Fine Double - ...,40ef98354335cf4780937da703ed6d65.jpg,SOMETHINC BROW WIZ Retractable Eyebrow,9cf798e5f940429f14b4af0fd48992a4.jpg
4,4,Goblin♛ COD Tas Ransel Anak Sekolah Karakter K...,c28512df97d0fc1d61fd30de966e01c3.jpg,TAS KARAKTER ANAK LUCU,a4c09a46d8b1adda43a2433d40bba583.jpg


In [12]:
d_train.isna().sum()

title_1    0
image_1    0
title_2    2
image_2    0
Label      0
dtype: int64

In [13]:
d_final.isna().sum()

pair_index    0
title_1       0
image_1       0
title_2       8
image_2       0
dtype: int64

In [14]:
d_final.loc[d_final.title_2.isna(), 'title_2'] = d_final.loc[d_final.title_2.isna(), 'title_1']

In [ ]:
d_final.isna().sum()

In [ ]:
d_train.Label.value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.savefig('../figures/label_dist.png')

## Need to cleansing

- lower case
- add space before|after symbol like ][
- add space after number

In [15]:
def text_cleansing(title):
    table = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
    
    title = title.lower()
    title = re.sub('(\d+)([a-zA-Z]+)', r'\1 \2', title)
    title = re.sub('\]', r' ] ', title)
    title = re.sub('\[', r' [ ', title)
    title = title.translate(table)
    title = re.sub(r'[^(a-z|A-Z|0-9)]', ' ', title)
    title = " ".join(title.split())
    
    return title

In [ ]:
d_train.dropna(inplace=True)

In [ ]:
d_train['title_1_pre'] = d_train.title_1.apply(text_cleansing)
d_train['title_2_pre'] = d_train.title_2.apply(text_cleansing)

In [ ]:
d_test['title_1_pre'] = d_test.title_1.apply(text_cleansing)
d_test['title_2_pre'] = d_test.title_2.apply(text_cleansing)

In [16]:
d_final['title_1_pre'] = d_final.title_1.apply(text_cleansing)
d_final['title_2_pre'] = d_final.title_2.apply(text_cleansing)

In [ ]:
d_train[["title_1_pre", "title_2_pre", "Label"]]

In [ ]:
d_test[["title_1_pre", "title_2_pre"]]

### Image

In [19]:
def read_image_size(path):
    try:
        img_path = os.path.join("../data/final/test_img/test_img", path)
        img_arr = Image.open(img_path)
        img_arr = img_arr.resize((224,224))
        img_arr = img_arr.convert('RGB')
        shape = img_arr.size
    except:
        shape = None
    return shape

In [20]:
import numpy as np
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
img_arr = read_image_size(d_train.loc[5253, 'image_2'])

In [ ]:
d_train['image_1_size'] = d_train.image_1.progress_apply(read_image_size)
d_train['image_2_size'] = d_train.image_2.progress_apply(read_image_size)

In [21]:
d_final['image_1_size'] = d_final.image_1.progress_apply(read_image_size)

100%|██████████| 32580/32580 [13:06<00:00, 41.40it/s]


In [23]:
d_final['image_2_size'] = d_final.image_2.progress_apply(read_image_size)

100%|██████████| 32580/32580 [12:33<00:00, 43.23it/s]


In [24]:
d_final.isna().sum()

pair_index      0
title_1         0
image_1         0
title_2         0
image_2         0
title_1_pre     0
title_2_pre     0
image_1_size    0
image_2_size    0
dtype: int64

In [ ]:
d_train.isna().sum()

In [ ]:
d_train[d_train.image_2_size.isna()]

In [ ]:
d_train["image_1_channel"] = d_train.image_1_size.apply(lambda x: x[2])
d_train["image_2_channel"] = d_train.image_2_size.apply(lambda x: x[2])

In [ ]:
d_train.image_1_channel.value_counts()

In [ ]:
d_train.image_2_channel.value_counts()

In [ ]:
d_train

In [ ]:
d_train.to_csv("../data/text_clean/train.csv", index=False)
d_test.to_csv("../data/text_clean/test.csv", index=False)

In [28]:
d_final[['pair_index', 'title_1', 'image_1', 'title_2', 'image_2', 'title_1_pre',
       'title_2_pre']].to_csv("../data/text_clean/final.csv", index=False)